# 对比各流域的农业用水压力

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import sys
sys.path.append("..")

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['axes.facecolor'] = 'white'

# 自定义配色
nature_colors = {
    'NS': "#c83c1c",
    'Nature': "#29303c",
    'NCC': "#0889a6",
    'NC': "#f1801f",
    'NG': "#006c43",
    'NHB': "#1951A0",
    'NEE': "#C7D530"
}

### Zhou老师用水量数据

In [2]:
# 周老师的用水数据
pns = pd.read_excel("../data/large/Zhou et al_2020_PNAS_dataset.xlsx", 1)

# 清洗数据的头部，让两行头部变一行
change_name_dic = {}
last_item = "None"
for col in pns:
    second_row = pns.loc[0, col]
    if "Unnamed" in col:
        change_name_dic[col] = last_item + ": " + second_row
    else:
        if type(second_row) is str:
            change_name_dic[col] = col + ": " + second_row
        last_item = col

pns.rename(change_name_dic, axis=1, inplace=True)
pns = pns.drop(0)

# 重命名表头，取消两边的空格
pns.rename({col: col.strip() for col in pns}, axis=1, inplace=True)

# 更改正确的数据类型
pns = pns.astype(float, errors='ignore')
pns['Year'] = pns['Year'].astype(int)
pns.iloc[:, 2:] = pns.iloc[:, 2:].astype(float)

pns.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/large/Zhou et al_2020_PNAS_dataset.xlsx'

周老师的数据是用各省的水资源利用调查+各省用水量公报用水量整合起来的用水量

假设水资源利用强度长期不变的基础上，按照经济指标进行的计算和拆分

因此在这里最原始的数据就是各个地级市（甚至只有省）的总用水量。

### 武旭同流域Shp数据+周老师数据 Identify

利用师兄给的 Shpfile 文件数据，加上周老师地级市的数据，使用GIS的 Identify 方法，为每个地级市标记它属于哪个一级流域。

需要查一下 Identify 的算法。

In [ ]:
identify = pd.read_excel("../data/private/perfectures_identity.xlsx", index_col=0)
identify.dropna().head()

In [ ]:
print("看看不同的流域，分别包含了多少个地级市，到时候好对比")
identify.groupby("MNAME_CH").count().iloc[:, 0].sort_values(ascending=False).head(10)

将周 PNAS 的数据和分属于哪个一级流域的数据整合到一起：

In [ ]:
merged_data = pd.merge(
    left=pns,
    right=identify,
    left_on='City_ID',
    right_on='Perfecture',
    right_index=False,
    left_index=False,
)

merged_data.head()
merged_data[merged_data['Year'] == 2000].groupby("MNAME_CH").count().iloc[:, 0].sort_values(ascending=False).head(10)

可以看到，整合后的数据和每个流域对应的数量还是一样的，说明数据结合的没有问题

In [ ]:
# 根据流域和年份来整合用水量
pivot_data = merged_data.pivot_table(
    index=['MNAME_CH', 'Year'],
    values=['Total water use', 'IRR', 'IND', 'RUR', 'URB'],
    aggfunc='sum',
)

pivot_data

In [ ]:
merged_data.to_csv("../data/large/pnas_identified.csv")

### 读图用水数据

In [ ]:
import os, sys

# 我们这里关心的是由统一的大河链接起来的一级流域，每个一级流域用最后一个最接近入海口的水文站来代表其入海前的实测径流量。
BASINS = {
    '淮河流域': '蚌埠吴家渡',
    '珠江流域': '高要',
    '长江流域': '大通',
    '海河流域': '海河闸',
    '黄河流域': '利津',
    '辽河流域': '六间房'
}

path = "../data/private/九大流域用水变化_读图/"

runoff = pd.DataFrame(index=np.arange(1950, 2020))
for basin in os.listdir(path):
    if basin in BASINS:
        file = os.path.join(path, basin, f"{basin}.xlsx")
        runoff[basin] = pd.read_excel(file, BASINS[basin], index_col='年份')['年平均径流量']
runoff = runoff.dropna(axis=0, how='all')
runoff

In [ ]:
total_wu = pd.DataFrame()
for basin in BASINS:
    total_wu[basin] = pivot_data.loc[basin, 'IRR'] * 10
    
total_wu.head()
total_wu.shape

In [ ]:
# 还原后的天然河川径流量
reducted = (runoff + total_wu).dropna(how='all')

ratio = pd.DataFrame()
# 计算用水量占天然河川径流量的比例
for basin in BASINS:
    ratio[basin] = total_wu[basin] / reducted[basin]
    
ratio.head()

可以看到，还需要使用地下水所占用水量的比例对用水量进行修正

## 使用地下水占比进行修正

使用水资源公报中近20年来地下水和地表水的使用占比，为每个一级流域过去用水的大致来源进行估计修正。

$$\alpha = \frac{WU_{surface}}{WU_{total}} $$

$$ pnas_{surface} = pnas_{total} * \alpha $$

其中 $WU$ 是来自水资源公报的用水数据，而 $pnas$ 是来自周老师数据的用水量。

In [ ]:
# 利用

water_source = pd.read_excel("../data/water_source.xlsx", index_col=0, header=1)
water_source_columns = pd.read_excel("../data/water_source.xlsx", index_col=0).columns.tolist()

alpha_dic = {}
for i in BASINS:
    abstract_name = i[:2]
    if abstract_name in water_source_columns:
        index = water_source_columns.index(abstract_name)
        surface, ground, others = water_source.iloc[:, index:index+3].mean().values
        alpha = surface / (surface + ground + others)
        alpha_dic[i] = alpha
    else:
        print(f"No matched name: {i}")
        
alpha_dic

In [ ]:
# 还原后的天然河川径流量
ENG_NAME = {
    '淮河流域': 'Huai', 
    '珠江流域': 'Zhu', 
    '长江流域': 'Chang', 
    '黄河流域': 'Yellow', 
    '海河流域': 'Hai', 
}

from tools.mk_test import mk_test
fig, ax = plt.subplots()

ratio = pd.DataFrame()
# 计算用水量占天然河川径流量的比例
for basin in ENG_NAME:
    wu = total_wu[basin] * alpha_dic[basin]
    reducted = (runoff[basin] + wu).dropna(how='all')
    tmp_ratio = wu / reducted
    ratio[basin] = tmp_ratio
    trend = mk_test(tmp_ratio.values)[0]
    ax.plot(total_wu.index, tmp_ratio, "-o", label="{}: {}".format(ENG_NAME[basin], trend))
    
ax.legend()
plt.show();

四种不同的类型：

- 压力增加，压力水平大的流域：黄河
- 压力增加，但远没有达到地表水资源压力的流域：长江、珠江
- 压力不变，压力水平大：淮河
- 压力不变，压力水平极限：海河